<a href="https://colab.research.google.com/github/PedroNishimura/project-ETL/blob/main/getCommentsIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas_gbq
import pandas as pd
import requests
from google.oauth2 import service_account
from google.colab import userdata

In [ ]:
apiKey = userdata.get('api_key')
channelId = 'UCSQ2vulanpm-UYOzbDdu60A'
url = 'https://youtube.googleapis.com/youtube/v3/commentThreads?part=snippet%2Creplies&allThreadsRelatedToChannelId={}&key={}'.format(channelId, apiKey)
df = pd.DataFrame()

SCOPES = [
    'https://www.googleapis.com/auth/cloud-plataform'
]
credentials = service_account.Credentials.from_service_account_info(
  {
    "type": "service_account",
    "project_id": "savvy-container-401123",
    "private_key_id": userdata.get('private_key_id'),
    "private_key": userdata.get('private_key'),
    "client_email": userdata.get('client_email'),
    "client_id": userdata.get('client_id'),
    "auth_uri": "https://accounts.google.com/o/oauth2/auth",
    "token_uri": "https://oauth2.googleapis.com/token",
    "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
    "client_x509_cert_url": "https://www.googleapis.com/robot/v1/metadata/x509/apibq-140%40savvy-container-401123.iam.gserviceaccount.com",
    "universe_domain": "googleapis.com"
  }

)

response = requests.get(url).json()
nextPageToken = response['nextPageToken']

while (nextPageToken != ''):
  try:
    response['items']
  except:
    try:
      nextPageToken = response['nextPageToken']
    except:
      nextPageToken = ''
    continue

  for element in response['items']:
    comment = pd.json_normalize(element['snippet']['topLevelComment']['snippet'])
    comment = comment[
      [
        'videoId',
        'textOriginal',
        'authorDisplayName',
        'likeCount',
        'publishedAt'
      ]
    ]
    df = df.append(comment)

  try:
    nextPageToken = response['nextPageToken']
  except:
    nextPageToken = ''

  url = 'https://youtube.googleapis.com/youtube/v3/commentThreads?part=snippet%2Creplies&pageToken={}&allThreadsRelatedToChannelId={}&key={}'.format(nextPageToken, channelId, apiKey)
  response = requests.get(url).json()

In [ ]:
pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = 'DataEngineerStudies'
pandas_gbq.to_gbq(df, 'testes.ipp', project_id='savvy-container-401123', if_exists='replace')